In [1]:
import os
import glob
import pickle
import pandas as pd
import numpy as np
from scipy.signal import medfilt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

In [2]:
import warnings

warnings.filterwarnings("ignore", message="Could not infer format, so each element")
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [3]:
cores = '65k_64_56'

BASE_DIR        = f"../data/MAR2_al/MAR2_hpl/{cores}/"   # point this at the “14” folder
DATA_DIR        = f"../results/{cores}/"

OUT_G           = f"../results/{cores}/G_matrix.csv"
OUT_T           = f"../results/{cores}/Ta_vector.csv"
OUT_P           = f"../results/{cores}/Pij.csv"
OUT_f           = f"../results/{cores}/fij.csv"

# Number of DVFS states to detect via clustering
DVFS_STATES     = 4

In [4]:
# --- 1. Load a sample MAR2_hpl round -------------------
def load_round_pkl(base_dir, round_name="round0"):
    """
    Load the roundX.pkl for a given HPL configuration
    """
    pkl_path = os.path.join(base_dir, round_name, "preprocessing", f"{round_name}.pkl")

    print(pkl_path)
    
    with open(pkl_path, "rb") as f:
        data = pickle.load(f)
    return data

def ensure_dirs():
    os.makedirs(DATA_DIR, exist_ok=True)

# ─────────────────────  PACKAGE → PER‑CORE POWER  ──────────────────────────

def safe_package_power(
    energy_df: pd.DataFrame,
    dt: float,
    rapl_units_j: float | None = None,
    modulus: int | None = None,
    max_pkg_watts: float = 400.0,
) -> pd.Series:
    """
    Convert monotonic RAPL energy counters -> package power (W) robustly.

    Parameters
    ----------
    energy_df : DataFrame (T×P)
        Raw cumulative energy counters (as read from MSR), one column per package.
    dt : float
        Sampling interval in seconds.
    rapl_units_j : float, optional
        If the raw counter units are *not* joules, pass the conversion factor.
        Example: Intel MSR_RAPL_UNIT gives 1 LSb = 15.3e-6 J by default.
        If None we assume the DataFrame is already in joules.
    modulus : int, optional
        Counter modulus for wrap‑around (e.g. 2**32 or 2**48).  If None,
        wrap detection is skipped.
    max_pkg_watts : float
        Any single‑sample power delta above this is treated as bad data and dropped.

    Returns
    -------
    Series length T
        Package power in Watts (sum over all package columns, cleaned).
    """
    E = energy_df.copy()

    # 1) optional unit conversion
    if rapl_units_j is not None:
        E *= rapl_units_j

    # 2) unwrap counters
    if modulus is not None:
        diff = E.diff()
        wrapped = diff < 0
        E[wrapped] += modulus * rapl_units_j if rapl_units_j else modulus

    diff = E.diff().fillna(0)
    P_pkg = diff.sum(axis=1) / dt  # combine packages -> Watts

    # 3) drop negative or implausibly large spikes
    P_pkg[(P_pkg < 0) | (P_pkg > max_pkg_watts)] = np.nan
    # interpolate small gaps, forward‑fill last resort
    P_pkg = P_pkg.interpolate(limit_direction="both")

    return P_pkg

def package_power_watts(data: dict, dt: float = 0.5) -> pd.Series:
    """Convert RAPL energy counters (J) → package power (W)."""
    pkg_erg = data["power_from_erg_pkg_cpu"]  # T×P energy (J)

    raw_E = data["power_from_erg_pkg_cpu"]      # T×P
    pkg_power_W = safe_package_power(
    raw_E,
    dt=0.5,                 # your sample period
    rapl_units_j=15.3e-6,   # or None if already Joules
    modulus=2**32           # adjust if 48‑bit
    )

    
    return pkg_power_W  # Series (T samples)

def per_core_power(
    pkg_power: pd.Series,
    data: dict,
    bar_p: float = 2.64,
    active_thresh: float = 0.5,
    med_k: int = 3,
) -> pd.DataFrame:
    """
    Split package power across *active* cores only (C0 residency > threshold).
    Adds the idle baseline `bar_p` to every core.
    Optionally median‑filters the result (`med_k` must be odd; 1 = no filter).
    """
    cores = data["temp_core"].columns
    c0 = data["C0_norm_tsc_core"] > active_thresh
    power = pd.DataFrame(index=pkg_power.index, columns=cores, dtype=float)

    for t in pkg_power.index:
        active = c0.loc[t]
        n_active = active.sum()
        if n_active == 0:
            power.loc[t] = bar_p
        else:
            residual = pkg_power.loc[t] - bar_p * len(cores)
            share = residual / n_active
            power.loc[t, active] = bar_p + share
            power.loc[t, ~active] = bar_p

    if med_k > 1:
        power = power.apply(medfilt, kernel_size=med_k)
    return power

def estimate_bar_p(data: dict, dt: float = 0.5, idle_thresh: float = 0.05) -> float:
    """
    Estimate per‑core idle power BAR_P (Watts) from roundX.pkl contents.

    Parameters
    ----------
    data : dict
        The dictionary loaded from roundX.pkl (contains 'power_from_erg_pkg_cpu',
        'C0_norm_tsc_core', etc.).
    dt : float
        Sampling period in seconds (default 0.5 s).
    idle_thresh : float
        Core is considered idle if C0 residency ≤ `idle_thresh`
        (e.g. ≤ 5 % of the 0.5‑s window).

    Returns
    -------
    bar_p : float
        Estimated idle power per core in Watts.
    """
    # 1) Package power in Watts (sum across packages)
    pkg_energy_J = data["power_from_erg_pkg_cpu"]      # shape T×P (Joules)
    pkg_power_W  = pkg_energy_J.diff().fillna(0).sum(axis=1) / dt  # Series length T

    # 2) Identify fully‑idle timestamps (no core in C0 above threshold)
    c0_res       = data["C0_norm_tsc_core"]            # T×N fraction of time in C0
    idle_mask    = (c0_res <= idle_thresh).all(axis=1) # Series length T (bool)

    if idle_mask.sum() == 0:
        raise RuntimeError("No fully‑idle samples found – choose higher idle_thresh.")

    # 3) Package idle power ➔ per‑core idle power
    Ncores   = c0_res.shape[1]
    pkg_idle = pkg_power_W[idle_mask].median()         # robust median Watts
    bar_p    = pkg_idle / Ncores

    return float(bar_p)


# ───────────────────────  FREQUENCY CLUSTERING  ────────────────────────────
def cluster_freq(
    freq_raw: pd.DataFrame, n_states: int = 4, random_state: int = 0
) -> tuple[pd.DataFrame, np.ndarray]:
    """
    Cluster per‑core normalized frequency into `n_states` bins.
    Returns (clustered_df, sorted_centers).
    """
    uniq = np.unique(freq_raw.values).reshape(-1, 1)
    km = KMeans(n_clusters=n_states, random_state=random_state).fit(uniq)
    centers = np.sort(km.cluster_centers_.flatten())

    freq_c = freq_raw.copy().astype(float)
    for col in freq_raw.columns:
        freq_c[col] = centers[km.predict(freq_raw[col].values.reshape(-1, 1))]
    return freq_c, centers


# ───────────────  DERIVE f_ij & P_ij  ──────────────────────────────────────
def derive_f_P(
    power_df: pd.DataFrame,
    freq_clustered: pd.DataFrame,
    freq_levels: np.ndarray,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Compute:
      f_ij = mean effective frequency (Hz) of core i in DVFS state j
      P_ij = mean power  (W)            of core i in DVFS state j
    """
    cores = power_df.columns
    f_ij = pd.DataFrame(index=cores, columns=freq_levels, dtype=float)
    P_ij = pd.DataFrame(index=cores, columns=freq_levels, dtype=float)

    for f in freq_levels:
        mask = freq_clustered == f           # True where core is in that state
        f_ij[f] = freq_clustered.where(mask).mean(axis=0)   # Hz
        P_ij[f] = power_df.where(mask).mean(axis=0)         # W
    return f_ij, P_ij


# ────────────────  THERMAL CONDUCTANCE (GS)  ───────────────────────────────
def estimate_GS(
    temp_df: pd.DataFrame,
    power_df: pd.DataFrame,
    med_k: int = 3,
) -> pd.DataFrame:
    """Return GS matrix (°C/W) using centered covariance / variance."""
    if med_k > 1:
        temp_df = temp_df.apply(medfilt, kernel_size=med_k)

    X = power_df.values - power_df.values.mean(0)
    Y = temp_df.values - temp_df.values.mean(0)
    T, N = X.shape

    var = (X ** 2).mean(0)
    cov = (X.T @ Y) / T
    GS = cov.T / var[np.newaxis, :]
    return pd.DataFrame(GS, index=temp_df.columns, columns=temp_df.columns)


# ───────────────  ONE‑SHOT PIPELINE  ───────────────────────────────────────
def process_round(
    base_dir: str,
    round_name: str = "round0",
    n_states: int = 4,
    dt: float = 0.5,
    med_k: int = 3,
    save_dir: str | None = None,
) -> dict:

    data  = load_round_pkl(base_dir, round_name)
    cores = data["temp_core"].columns

    pkg_pw = package_power_watts(data, dt)
    bar_p  = estimate_bar_p(data, dt)
    p_core = per_core_power(pkg_pw, data, bar_p=bar_p, med_k=med_k)

    # ---- use C0_norm_tsc_freq_core as the raw performance signal ----
    freq_raw                 = data["C0_norm_tsc_freq_core"]
    freq_c, freq_centers     = cluster_freq(freq_raw, n_states)

    # derive f_ij from frequency, P_ij from power
    f_ij, P_ij = derive_f_P(
        power_df       = p_core,
        freq_clustered = freq_c,
        freq_levels    = freq_centers,
    )

    GS = estimate_GS(data["temp_core"], p_core, med_k=med_k)
    Ta = data["temp_core"].min(axis=0)

    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        GS.to_csv(f"{save_dir}/G_matrix.csv", index=False)
        P_ij.to_csv(f"{save_dir}/Pij.csv", index=False)
        f_ij.to_csv(f"{save_dir}/fij.csv", index=False)
        pd.DataFrame(Ta).to_csv(f"{save_dir}/Ta_vector.csv", header=False)

    return {
        "GS": GS,
        "P_ij": P_ij,
        "f_ij": f_ij,     # now measured in Hz
        "Ta": Ta,
        "bar_p": bar_p,
        "power_core": p_core,
        "freq_clustered": freq_c,
        "centers": freq_centers,
    }

In [5]:
# ─── Example usage ─────────────────────────────────────────────────────────
if __name__ == "__main__":  # run a quick sanity test
    result = process_round(BASE_DIR, n_states=DVFS_STATES, save_dir=DATA_DIR)

../data/MAR2_al/MAR2_hpl/65k_64_56/round0/preprocessing/round0.pkl


In [6]:
result['bar_p']

-0.018233163016183034

In [7]:
print("P_ij diag median:", np.median(np.diag(result["P_ij"])))

P_ij diag median: 1.4249106724011767e-06


In [8]:
print("GS diag median (°C/W):", np.median(np.diag(result["GS"])))

GS diag median (°C/W): 580.6516689348969


In [9]:
print(result["GS"].describe().loc[['min','mean','max']])

             0           1           2           3           4           5    \
min   367.429362  445.589474  377.148780  409.252728  428.791435  409.252728   
mean  563.241349  628.836599  574.767527  599.714108  615.267213  599.714108   
max   685.685637  763.565452  698.983701  730.718797  748.643628  730.718797   

             6           7           8           9    ...         102  \
min   409.252728  409.252728  377.148780  409.252728  ...  367.429362   
mean  599.714108  599.714108  574.767527  599.714108  ...  563.241349   
max   730.718797  730.718797  698.983701  730.718797  ...  685.685637   

             103         104         105         106         107         108  \
min   367.429362  367.429362  367.429982  367.429362  367.429362  367.429362   
mean  563.241349  563.241349  563.242024  563.241349  563.241349  563.241349   
max   685.685637  685.685637  685.686396  685.685637  685.685637  685.685637   

             109         110         111  
min   367.429362  367.